## Notebook For Eclipse Software Deployment ##
<i>Written by David Story </i>

___
### Description
This notebook was written to be an easy-to-use plateform for deploying and implementing software to AWS cloud computing services and creating datasets for the Eclipse Megamovie project, as well as related computationally heavy research projects.

This notebook will automate a number of tasks for preparing your AWS EC2 instances and deploying software to those instances. 

This notebook will:

- Create a specified amount of instances
- Send datasets to those instances
- Send processing jobs and parameters to instances
- Run jobs
- Monitor jobs
- Retrieve and store results
___
### Information on Provided Files ###
This repository contains the following folders:

   - examples
   - keys
   - documentation
   - images
   - movies
   - logs
   - figures
   - results

___
#### examples: ####
This folder provides example files that will allow you to understand how files are being fed to this notebook, and how they are formated, to help you understand how to generate your files to allow this notebook to run smoothly.

#### keys: ####
This folder should hold your ssh or .pem keys that you generated for your EC2 instance

#### documentation: ####
This should should hold the .csv that describes your EC2 instance names and attributes.

#### images: ####
This should hold any images generated.

#### movies: ####
This folder should hold any movies generated.

#### servers: ####
This folder should hold any logs generated relating to server performance, states, and program performance and states.

#### figures: ####
This folder should hold any figures generated.

#### results: ####
This folder should hold any results generated.

___
### Dependencies
This notebook requires the following dependenices

   - boto3
   - botocore
   - jmespath


___
### Setting up AWS Config

We will be using the Boto3 API to interface Python with AWS, before we can do that we need to instance the AWS CLI from here:

https://aws.amazon.com/cli/

After you have installed the CLI, you must configure your AWS credentials on the machine that you will run this code on, the configuration process can be found here:

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration

<b>Ensure that in the configure step you set your region to the region that your EC2 and S3 instances and buckets are at, else you will not be able to access them with the API</b> 

___

### Viewing your current EC2 Instances###

Using the Boto3 library, we are able to acquire information about our instances that are in AWS using this Python API, below we are importing the libraries we will need:

In [1]:
import sys
import boto3
from botocore.exceptions import ClientError

Now that the libraries are acquired, we will use the ec2 resource to see what instances we currently have on our console:

In [2]:
# variable for ec2 resource
ec2 = boto3.resource('ec2')
client = boto3.client('ec2')

# printing what running instances we have up
print("EC2 Instances Running:")

# filtering for instances by name and if running
running_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

# printing the names of running instances and their machine type
for instance in running_instances:
    print(instance.id, instance.instance_type)

# printing what stopped instances we have
print("\nEC2 Instances Stopped:")

# filtering for instances by name and if stopped
stopped_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['stopped']}])

# printing the names of running instances and their machine type
for instance in stopped_instances:
    print(instance.id, instance.instance_type)

# for other cases other than running or stopped
other_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['pending', 'shutting-down',
                                                    'terminated','stopping']}])

print("\nEC2 Instances Other:")
for instance in other_instances:
    print(instance.id, instance.instance_type)
    
print("\nDone Searching")

EC2 Instances Running:

EC2 Instances Stopped:
i-0d606940d0ad53cd1 t2.micro
i-06413b62ffcd07c4d t2.micro
i-03a2ab946f0ac123e t2.micro

EC2 Instances Other:

Done Searching


### Setting up the instances needed ###

We will now be preparing the instances that we want to process the images with.

<b> Warning: in the following process we will be stopping all of the current instances and creating a defined amount of instances that we will use for our image processing </b>

You will be asked to confirm that you want to begin the process of pausing your instances and starting the server process.

In [3]:
# Waits for user to authorize stop of their instances
go = False
while(go == False):
    user = input("Authorize to stop and store current instances ([Y]/n): ")
    if (user == "Y"):
        go = True
    elif (user == 'n'):
        print("Entered No: Exiting Program")
        sys.exit()
    else:
        print("Invalid Input:", user)

# Filters a list of all instances before we create new ones
old_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 
              'Values': ['running', 'stopped',
                        'pending','shutting-down'
                         ,'terminated','stopping']
             }])

# Stores ID of old instances and then stops all instances
print("Stopping:")
stopIds = []
for instance in old_instances:
    instance.stop(instance.id)
    stopIds.append(instance.id)
print(stopIds)    

Authorize to stop and store current instances ([Y]/n): Y
Stopping:
['i-0d606940d0ad53cd1', 'i-06413b62ffcd07c4d', 'i-03a2ab946f0ac123e']


### Creating new instances ###

Now that the above instances are stopped, we want to create the number of instances that we want to use. AWS by default only allows you to run 20 instances in reserve, with a more limited number on On-Demand time. This notebook is set to the 20 instance reserved limit. 

You can request more instance allocations from a link on this page: https://aws.amazon.com/ec2/faqs/#How_many_instances_can_I_run_in_Amazon_EC2

Then change the 20 on line 6 to the maximum number of instances you can use.

Input the number of instances you want to use below:

In [4]:
answer = True
while answer:
    number_of_instances = abs(int(input("Number of instances to use: ")))
    if (number_of_instances <= 0):
        print("Invalid: Enter valid number")
    elif (number_of_instances > 20):
        print("Invalid: Cannot create more than 20 Instances")
    else:
        answer = False
        
print(number_of_instances, "EC2 instances will be created.")

Number of instances to use: 3
3 EC2 instances will be created.


We will now create a key pair that will use to SSH into our servers. We will share the same key with every server for convenience.

In [5]:
# Creating key pair for Eclipse Megamovie usages
try:
    ec2.create_key_pair(KeyName='EMkey')
except:
    print("Key already created or invalid.")

Key already created or invalid.


We will create a list of names for our new EC2 Instances

In [6]:
base = "ECMEGA-SERVER-"
namelist = []
for name in range(number_of_instances):
    newname = base + str(name+1)
    namelist.append(newname)
namelist

['ECMEGA-SERVER-1', 'ECMEGA-SERVER-2', 'ECMEGA-SERVER-3']

We will next find our avaliable security groups:

In [10]:
secureGroups = ec2.security_groups.all()
print("Avaliable Security Groups:")
val = 0
for groups in secureGroups:
    val += 1
    print(str(val) +".",groups)

Avaliable Security Groups:
1. ec2.SecurityGroup(id='sg-015d5ba79fb259b72')
2. ec2.SecurityGroup(id='sg-023aed919f878ebbe')
3. ec2.SecurityGroup(id='sg-03563349601f6c0a8')
4. ec2.SecurityGroup(id='sg-03cd3639298cadbc7')
5. ec2.SecurityGroup(id='sg-043d0b3e381abac9d')
6. ec2.SecurityGroup(id='sg-04aad0e0fdad5d76a')
7. ec2.SecurityGroup(id='sg-050efbbde77ac2fb4')
8. ec2.SecurityGroup(id='sg-0631d2aaba9aaa914')
9. ec2.SecurityGroup(id='sg-072bb029b1e3a974e')
10. ec2.SecurityGroup(id='sg-07dcb7d6071a2c647')
11. ec2.SecurityGroup(id='sg-097b8f2d123cf007a')
12. ec2.SecurityGroup(id='sg-0fc4818efcc8ed55d')
13. ec2.SecurityGroup(id='sg-ca8a2aa6')
